In [ ]:
import sys
sys.path.append('../')

import numpy as np
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy
import cartopy.crs as ccrs
from geometry import Geometry
from forcing import Forcing
from plotfunctions import add_lonlat

np.seterr(all='ignore')
%matplotlib inline
#%matplotlib widget
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
cmap1 = mpl.colors.ListedColormap(plt.get_cmap('cmo.deep_r')(np.linspace(0,.4,255)))
#cmap2 = 'cmo.deep'
#cmap2 = 'gist_earth_r'
cmap2 = 'cmo.rain_r'
cmap3 = mpl.colors.ListedColormap(plt.get_cmap('cmo.gray')(np.linspace(.4,.7,255)))

#lon0,lon1 = 245.8,251
#lat0,lat1 = -75.2,-74.4

#proj = ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0)#260-360)


In [ ]:
plt.style.use('style_paper')

mpl.rc('figure.subplot',left=.1,right=.95,top=.9,bottom=0,wspace=.1,hspace=.15)

fig = plt.figure(figsize=(7,6))

#LADDIE model placeholder
"""
ax = fig.add_subplot(221)
ax.set_axis_off()
ax.set_title('a) LADDIE model',loc='left')

ax = fig.add_subplot(663)
ax.set_axis_off()

#Antarctic map

ds = xr.open_dataset('../../../data/BedMachineAntarctica_2020-07-15_v02.nc')
ds = ds.coarsen(x=50,y=50,boundary='trim').median()
ds = add_lonlat(ds)

bed = ds.bed
bed = np.where(ds.lat<-60,ds.bed,np.nan)
im1 = ax.pcolormesh(ds.x,ds.y,bed,cmap=cmap1,shading='nearest',vmin=-1500,vmax=0)
im2 = ax.pcolormesh(ds.x,ds.y,np.where(ds.mask==0,np.nan,ds.thickness),cmap=cmap2,shading='nearest',vmin=0,vmax=2000)
var = np.where(ds.mask==0,np.nan,ds.bed)
var = np.where(ds.mask==3,np.nan,var)
im3 = ax.pcolormesh(ds.x,ds.y,var,cmap=cmap3,shading='nearest',vmin=-2000,vmax=0)

ax.text(.15,.33,'b)',transform=ax.transAxes,c='w')
ax.text(.27,.65,'c)',transform=ax.transAxes,c='w')
"""
#Crosson Dotson

ax = fig.add_axes([.1,.4,.3,.5],projection=ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0))
#ax = fig.add_subplot(221,projection=ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0))
ax.set_axis_off()
ax.set_title('a) Crosson-Dotson geometry',loc='left')

x0,x1,y0,y1 = 3000,4000,7500,8500
ds = xr.open_dataset('../../../data/BedMachineAntarctica_2020-07-15_v02.nc')
ds = ds.isel(x=slice(x0,x1),y=slice(y0,y1))
ds = add_lonlat(ds)

im1 = ax.pcolormesh(ds.lon,ds.lat,ds.bed,cmap=cmap1,shading='nearest',transform=ccrs.PlateCarree(),vmin=-1500,vmax=0)
im2 = ax.pcolormesh(ds.lon,ds.lat,np.where(ds.mask==0,np.nan,ds.surface-ds.thickness),cmap=cmap2,shading='nearest',transform=ccrs.PlateCarree(),vmin=-2000,vmax=0)
var = np.where(ds.mask==0,np.nan,ds.bed)
var = np.where(ds.mask==3,np.nan,var)
im3 = ax.pcolormesh(ds.lon,ds.lat,var,cmap=cmap3,shading='nearest',transform=ccrs.PlateCarree(),vmin=-2000,vmax=0)

ax.set_extent([360-114,360-109,-75.3,-74.3],crs=ccrs.PlateCarree())
gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,color='.5', dms=True, x_inline=False, y_inline=False)

#gl.bottom_labels=False
gl.right_labels=False
gl.xlabel_style = {'color': 'gray'}
gl.ylabel_style = {'color': 'gray'}

ax.text(.85,.17,'Kohler Gl.',rotation=0,transform=ax.transAxes,c='k')
#ax.text(.2,.15,'Dotson Channel',rotation=30,transform=ax.transAxes)
ax.text(.1,.53,'Bear Peninsula',transform=ax.transAxes,c='k')
ax.text(.87,.2,'Smith\nWest Gl.',rotation=-35,transform=ax.transAxes,c='k')
ax.text(.85,.42,'Smith\nEast Gl.',rotation=10,transform=ax.transAxes,c='k')
ax.text(.8,.58,'Pope Gl.',rotation=15,transform=ax.transAxes,c='k')
ax.text(.4,.72,'Crosson\nIce Shelf',rotation=0,transform=ax.transAxes)
ax.text(.28,.22,'Dotson\nIce Shelf',rotation=0,transform=ax.transAxes)

#FRIS
ax = fig.add_axes([.5,.4,.43,.5],projection=ccrs.SouthPolarStereo(true_scale_latitude=-78,central_longitude=-20))
#ax = fig.add_subplot(222,projection=ccrs.SouthPolarStereo(true_scale_latitude=-78,central_longitude=-20))
ax.set_axis_off()
ax.set_title('b) Filchner-Ronne geometry',loc='left')

x0,x1,y0,y1 = 3000,6500,3500,7000
ds = xr.open_dataset('../../../data/BedMachineAntarctica_2020-07-15_v02.nc')
#ds = ds.coarsen(x=10,y=10,boundary='trim').mean()
ds = ds.isel(x=slice(x0,x1),y=slice(y0,y1))
ds = add_lonlat(ds)

im1 = ax.pcolormesh(ds.lon,ds.lat,ds.bed,cmap=cmap1,shading='nearest',transform=ccrs.PlateCarree(),vmin=-1500,vmax=0)
im2 = ax.pcolormesh(ds.lon,ds.lat,np.where(ds.mask==0,np.nan,ds.surface-ds.thickness),cmap=cmap2,shading='nearest',transform=ccrs.PlateCarree(),vmin=-2000,vmax=0)
var = np.where(ds.mask==0,np.nan,ds.bed)
var = np.where(ds.mask==3,np.nan,var)
im3 = ax.pcolormesh(ds.lon,ds.lat,var,cmap=cmap3,shading='nearest',transform=ccrs.PlateCarree(),vmin=-2000,vmax=0)

ax.set_extent([275,332,-80,-77],crs=ccrs.PlateCarree())#
#ax.set_extent([273,330,-80.5,-76.0],crs=ccrs.PlateCarree())
gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,color='.5', dms=True, x_inline=False, y_inline=False)

#gl.bottom_labels=False
gl.left_labels=False
gl.xlabel_style = {'color': 'gray'}
gl.ylabel_style = {'color': 'gray'}

ax.text(.05,.32,'Evans I.S.',transform=ax.transAxes,color='k',ha='right',clip_on=False)
ax.text(.08,.13,'Carlson\nInlet',rotation=10,transform=ax.transAxes,color='k',ha='right',clip_on=False)
ax.text(.0,.10,'Rutford I.S.',transform=ax.transAxes,color='k',ha='center',clip_on=False)
ax.text(.37,-.04,'Institute\nI.S.',rotation=-45,transform=ax.transAxes,color='k')
ax.text(.59,.0,'Möller\nI.S.',rotation=60,transform=ax.transAxes,color='k',ha='center')
ax.text(.78,.0,'Foundation\nI.S.',rotation=-80,transform=ax.transAxes,ha='right',color='k')
ax.text(.75,.12,'Support\nForce Gl.',rotation=-80,transform=ax.transAxes,color='k')
ax.text(.90,.45,'Recovery\nGl.',transform=ax.transAxes,color='k',clip_on=False)
ax.text(.94,.66,'Slessor Gl.',rotation=15,transform=ax.transAxes,color='k',clip_on=False)
ax.text(.95,.75,'Bailey I.S.',rotation=80,transform=ax.transAxes,color='k',clip_on=False)

ax.text(.06,.24,'Fowler Peninsula',rotation=10,transform=ax.transAxes,color='k',ha='center',clip_on=False)
ax.text(.68,.52,'Berkner\nIsland',ha='center',transform=ax.transAxes,color='k')
ax.text(.3,.27,'K.I.R.',rotation=-80,transform=ax.transAxes,color='k')
#ax.text(.43,.26,'D.I.R.',transform=ax.transAxes,color='w')
ax.text(.48,.29,'H.I.R.',rotation=-60,transform=ax.transAxes,color='k')

ax.text(.25,.6,'Ronne Ice Shelf',transform=ax.transAxes)
ax.text(.78,.58,'Filchner\nIce Shelf',transform=ax.transAxes)


#CD forcing
geom = Geometry('CrossDots')
geom = geom.create()
forc = Forcing(geom).tanh(ztcl=500,Tdeep=.4,z1=250)

ax = fig.add_axes([.1,.1,.1,.25])
ax.set_title('c) Crosson-Dotson forcing',loc='left')
ax.plot(forc.Tz,forc.z)
ax.set_ylim([-2000,0])
ax.set_ylabel('Depth [m]')
ax.set_xlabel(r'Temperature [\textdegree C]')

ax = fig.add_axes([.25,.1,.1,.25])
ax.plot(forc.Sz,forc.z)
ax.set_ylim([-2000,0])
ax.set_yticklabels([])

#FR forcing
geom = Geometry('FRIS')
geom = geom.create()
forc = Forcing(geom).linear(S1=34.8,T1=-2.3)
ax.set_xlabel('Salinity [psu]')


ax = fig.add_axes([.5,.1,.1,.25])
ax.set_title('d) Filchner-Ronne forcing',loc='left')
ax.plot(forc.Tz,forc.z)
ax.set_ylim([-2000,0])
ax.set_yticklabels([])
#ax.set_ylabel('Depth [m]')
ax.set_xlabel(r'Temperature [\textdegree C]')

ax = fig.add_axes([.65,.1,.1,.25])
ax.plot(forc.Sz,forc.z)
ax.set_ylim([-2000,0])
ax.set_yticklabels([])
ax.set_xlabel('Salinity [psu]')

ax = fig.add_axes([.7,.1,.15,.25])
ax.set_axis_off()
cb = plt.colorbar(im2,ax=ax)
cb.set_label('Ice shelf draft [m]')

plt.savefig('../../figures/draftplot_overview.png',dpi=450)